### Code Generator
The requirement: use a Frontier model to generate high performance C++ code from Python code

In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown,display,update_display
import gradio as gr
import subprocess

In [ ]:
## CHECK GEMINI API CONNECTIVITY

import google.generativeai as genai
import os
load_dotenv(override=True)
# Make sure your Gemini API key is set in your environment
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    print("GEMINI_API_KEY not found in environment variables.")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    try:
        model = genai.GenerativeModel("gemini-2.5-flash-lite")
        response = model.generate_content("Hello Gemini, are you working?")
        print("Gemini API connectivity successful!")
        print("Response:", response.text)
    except Exception as e:
        print("Gemini API connectivity failed:", e)

Gemini API connectivity successful!
Response: Yes, I am working! I'm a large language model, trained by Google. I'm ready and available to help you with whatever you need.

What can I assist you with today?


In [5]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')

In [6]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
gemini = genai.GenerativeModel()
OPENAI_MODEL = "gpt-4o"
GEMINI_MODEL = "gemini-2.5-flash-lite"

In [7]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [8]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [10]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [11]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

def optimize_gemini(python):
    stream = openai.chat.completions.create(model=GEMINI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)


In [19]:
def optimize_gemini(python):
    gemini = genai.GenerativeModel("gemini-2.5-flash-lite")
    prompt = user_prompt_for(python)
    response = gemini.generate_content(prompt)
    cpp_code = response.text
    print(cpp_code)
    write_output(cpp_code)

In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.411320 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Calculate function in C++
double calculate(long iterations, long param1, long param2) {
    double result = 1.0;
    for (long i = 1; i <= iterations; ++i) {
        long j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    // Start timing
    auto start_time = std::chrono::high_resolution_clock::now();

    // Calculate result
    double result = calculate(100000000, 4, 1) * 4;

    // End timing
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;

    // Output result and execution time with specified precision
    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << elapsed.count() << " seconds" << std::endl;

    return 0;


In [16]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.859081 seconds


#### Compiling C++ and executing
This next cell contains the command to compile a C++ file on my M1 Mac.
It compiles the file optimized.cpp into an executable called optimized
Then it runs the program called optimized

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below. If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!

Not an exact comparison but you can still get the idea of performance difference. For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [17]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.114756 seconds


In [24]:
optimize_gemini(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

int main() {
    // Use long double for higher precision, suitable for the calculations.
    long double result = 1.0;
    
    // Define parameters as long long to avoid potential overflow in intermediate calculations.
    long long iterations = 100000000LL;
    long long param1 = 4LL;
    long long param2 = 1LL;

    // Record start time using chrono for high-resolution timing.
    auto start_time = std::chrono::high_resolution_clock::now();

    // Optimized loop to reduce function call overhead and potential compiler optimizations.
    // The loop structure and calculations are directly translated.
    for (long long i = 1; i <= iterations; ++i) {
        // Calculate j using long long to prevent overflow.
        long long j1 = i * param1 - param2;
        result -= (1.0L / j1); // Use 1.0L for long double literal.
        
        long long j2 = i * param1 + param2;
        result += (1.0L / j2);
    }

    // Multi

In [25]:
# Repeat for Gemini - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.127710 seconds


In [26]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [31]:
def stream_gemini(python):
    gemini_model = genai.GenerativeModel(GEMINI_MODEL)
    prompt = user_prompt_for(python)
    result = gemini_model.generate_content(prompt, stream=True)
    reply = ""
    for chunk in result:
        text = getattr(chunk, "text", str(chunk))
        reply += text
        yield reply.replace('```cpp\n','').replace('```','')

In [29]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far   

In [32]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [33]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [34]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [35]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [38]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

## Gemini: 
Result: 3.141592658590
Execution Time: 0.047515 seconds

## Chatgpt
Result: 3.141592658590
Execution Time: 0.055739708000 seconds